# A full stock trading bot

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
serp_api_key = os.getenv("SERP_API_KEY")
perplexity_api_key = os.getenv("PEPLEXITY_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

if serp_api_key:
    print(f"serp_api_key exists and begins {serp_api_key[:8]}")
else:
    print("serp_api_key not set")

if perplexity_api_key:
    print(f"perplexity_api_key exists and begins {perplexity_api_key[:8]}")
else:
    print("perplexity_api_key not set")

# GPT 모델 선언
openai = OpenAI()
MODEL = 'gpt-4o-mini'

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBU
serp_api_key exists and begins c3ee9cec
perplexity_api_key exists and begins pplx-r4f


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
investing_site = Website("https://investing.com")
investing_site.links










['https://www.investing.com/',
 '/pro/pricing',
 '//uk.investing.com',
 '//in.investing.com',
 '//ca.investing.com',
 '//au.investing.com',
 '//za.investing.com',
 '//ph.investing.com',
 '//ng.investing.com',
 '//de.investing.com',
 '//es.investing.com',
 '//mx.investing.com',
 '//fr.investing.com',
 '//it.investing.com',
 '//nl.investing.com',
 '//pl.investing.com',
 '//pt.investing.com',
 '//br.investing.com',
 '//ru.investing.com',
 '//tr.investing.com',
 '//sa.investing.com',
 '//gr.investing.com',
 '//se.investing.com',
 '//fi.investing.com',
 '//il.investing.com',
 '//jp.investing.com',
 '//kr.investing.com',
 '//cn.investing.com',
 '//hk.investing.com',
 '//id.investing.com',
 '//ms.investing.com',
 '//th.investing.com',
 '//vn.investing.com',
 '//hi.investing.com',
 '/pro/pricing',
 'https://www.investing.com/markets/',
 'https://www.investing.com/indices/',
 'https://www.investing.com/indices/indices-futures',
 'https://www.investing.com/indices/major-indices',
 'https://www.i

In [15]:
print(investing_site.links[50:])
print(len(investing_site.links))

['https://www.investing.com/indices/eu-stoxx50', 'https://www.investing.com/indices/japan-ni225', 'https://www.investing.com/equities/', 'https://www.investing.com/stock-screener/', 'https://www.investing.com/equities/trending-stocks', 'https://www.investing.com/pro/propicks', 'https://www.investing.com/equities/united-states', 'https://www.investing.com/equities/pre-market', 'https://www.investing.com/equities/after-hours', 'https://www.investing.com/earnings-calendar/', 'https://www.investing.com/equities/americas', 'https://www.investing.com/equities/europe', 'https://www.investing.com/equities/52-week-high', 'https://www.investing.com/equities/52-week-low', 'https://www.investing.com/equities/most-active-stocks', 'https://www.investing.com/equities/top-stock-gainers', 'https://www.investing.com/equities/top-stock-losers', 'https://www.investing.com/equities/most-undervalued', 'https://www.investing.com/equities/most-overvalued', 'https://www.investing.com/equities/world-adrs', 'htt

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/01/23/ll

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nari-labs/Dia-1.6B',
 '/Qwen/Qwen3-235B-A22B',
 '/Qwen/Qwen3-30B-A3B',
 '/deepseek-ai/DeepSeek-Prover-V2-671B',
 '/sand-ai/MAGI-1',
 '/models',
 '/spaces/nari-labs/Dia-1.6B',
 '/spaces/enzostvs/deepsite',
 '/spaces/Qwen/Qwen3-Demo',
 '/spaces/stepfun-ai/Step1X-Edit',
 '/spaces/nvidia/describe-anything-model-demo',
 '/spaces',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets/OpenGVLab/InternVL-Data',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/Anthropic/values-in-the-wild',
 '/datasets/Eureka-Lab/PHYBench',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffusers'

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community discussion page',
   'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
nari-labs/Dia-1.6B
Updated

In [16]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include detaails of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nnari-labs/Dia-1.6B\nUpdated\n3 days ago\n•\n90k\n•\n1.56k\nQwen/Qwen3-235B-A22B\nUpdated\n1 day ago\n•\n10.1k\n•\n517\nQwen/Qwen3-30B-A3B\nUpdated\nabout 7 hours ago\n•\n9.34k\n•\n318\ndeepseek-ai/DeepSeek-Prover-V2-671B\nUpdated\n3 minutes ago\n•\n301\nsand-ai/MAGI-1\nUpdated\nabout 3 hours ago\n•\n508\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n965\n965\nDia 1.6B\n👯\nGenerate realistic di

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("Microsoft", "https://www.microsoft.com/ko-kr/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.microsoft.com/en-us/about'}, {'type': 'careers page', 'url': 'https://careers.microsoft.com/'}, {'type': 'company news', 'url': 'https://news.microsoft.com/'}, {'type': 'diversity page', 'url': 'https://www.microsoft.com/en-us/diversity/'}, {'type': 'sustainability page', 'url': 'https://www.microsoft.com/en-us/sustainability/'}]}


# Microsoft Company Brochure

---

## About Microsoft

### Your Partner in Innovation

Microsoft is a leading technology company dedicated to empowering individuals and organizations through advanced software and hardware solutions. With a diverse portfolio that includes Microsoft 365, Windows, Surface, Xbox, and Azure, Microsoft is at the forefront of the digital revolution.

---

## Our Products

- **Microsoft 365**: A comprehensive suite of productivity applications and cloud services designed to enhance collaboration and productivity.
- **Windows**: The world's most widely used operating system, providing a robust platform for all types of devices.
- **Surface**: Innovative devices designed for both personal and professional use, combining the versatility of tablets with the power of laptops.
- **Xbox**: A premier gaming platform that includes gaming hardware, software, and online services, fostering a community of gaming enthusiasts.
- **Azure**: A leading cloud services platform that enables businesses to build, deploy, and manage applications through Microsoft-managed data centers.

---

## Company Culture

### Empowering Employees and Fostering Innovation

At Microsoft, our culture is built on collaboration, diversity, and inclusion. We believe that the best ideas come from diverse perspectives and backgrounds. Our work environment is designed to inspire innovation and creativity, fostering a sense of belonging among all employees. We support continuous learning and professional growth, ensuring that our team stays ahead in the ever-evolving tech landscape.

---

## Our Customers

### Serving a Global Community

Microsoft serves a wide range of customers, including individuals, small businesses, and large enterprises worldwide. We are committed to understanding the needs of our customers and providing tailored solutions that drive success and improve lives.

---

## Careers at Microsoft

### Join Us in Creating the Future

Microsoft is always on the lookout for talented individuals to join our dynamic team. We offer a variety of career paths across multiple disciplines including software development, engineering, sales, marketing, and customer support. Our commitment to employee growth and community involvement helps us create a workplace that challenges and rewards our associates.

- **Learn & Grow**: Empowerment through continuous learning opportunities and professional training.
- **Diversity & Inclusion**: A place for everyone, where all voices are heard and valued.
- **Community Engagement**: An opportunity to make an impact through various initiatives and volunteer programs.

---

Join us in transforming the world through innovative technology at Microsoft! For more information, visit our website or reach out to learn about career opportunities, our latest products, or how we can assist your business.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True # 
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}]}


# Hugging Face Brochure

---

## Welcome to Hugging Face

**The AI community building the future.** At Hugging Face, we are on a mission to democratize machine learning, fostering collaboration among developers, researchers, and organizations. Our platform enables seamless interaction and innovation across models, datasets, and applications.

---

## Company Overview

- **Founded:** 2016
- **Vision:** To create an open environment for machine learning enthusiasts to collaborate and share knowledge.
- **Core Values:** Openness, Collaboration, Innovation, Community.

## What We Offer

### Explore Our Platform
- **1M+ Models:** Browse, collaborate, and deploy various state-of-the-art machine learning models.
- **250k+ Datasets:** Access an extensive collection of datasets to train and validate models.
- **AI Applications:** Discover numerous applications built by the community to address diverse challenges.

### Collaboration Opportunities
- Join a dynamic community to **create, discover**, and **collaborate** on machine learning better than ever before.

### Enterprise Solutions
Hugging Face provides robust enterprise solutions that ensure:
- **Security:** Enterprise-grade security features and access controls.
- **Support:** Dedicated support to accelerate your machine learning projects.
  
> **Starting at:** $20/user/month

---

## Our Customers

More than **50,000 organizations** trust Hugging Face, including:
- **Tech Giants:** Google, Microsoft, Amazon
- **Innovative Startups:** Grammarly, Writer
- **Research Institutions:** AI2

---

## Company Culture

Hugging Face embodies a culture of:
- **Innovation:** Encouraging creativity and experimentation.
- **Community:** Building relationships across diverse backgrounds and expertise.
- **Inclusivity:** Creating a welcoming environment for all.

Our team consists of over **212 members**, each contributing unique skills and perspectives to our collaborative mission.

---

## Careers at Hugging Face

Hugging Face is growing! We invite you to bring your talent and passion to our team. We offer:
- **Remote Work Options:** Work from anywhere in a flexible environment.
- **Impact-Driven Roles:** Contribute to cutting-edge projects in AI and machine learning.
- **Professional Development:** Opportunities for learning, growth, and collaboration with top experts in the field.

**Join us in redefining the future of AI!**

### Current Openings
Explore available positions on our [Jobs Page](https://huggingface.co/jobs).

---

## Connect with Us

Stay updated, connect, and collaborate with the Hugging Face community!
- **Website:** [Hugging Face](https://huggingface.co)
- **GitHub:** [Hugging Face GitHub](https://github.com/huggingface)
- **Twitter:** [@huggingface](https://twitter.com/huggingface)
- **Discord:** Join Our Channel!

---

**Together, we can build the future of AI!**

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>